# AI Agent Development

## Prerequisites

### Required Dotnet Packages for AI Agent Development

In [ ]:

#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"
#r "nuget: Microsoft.Extensions.AI, 9.10.0"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.10.0"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.Identity"
#r "nuget: System.Text.Json, 8.0.0"

In [ ]:
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Newtonsoft.Json, 13.0.3"

In [ ]:
using System;
using System.IO;
using System.ComponentModel;
using System.ClientModel;
using System.Text.Json;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;

using OpenAI;
using OpenAI.Chat;

using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using Newtonsoft.Json;


### Load config

In [ ]:
using DotNetEnv;
Env.Load("./.env");

In [ ]:
var endpoint = Environment.GetEnvironmentVariable("ENDPOINT") ?? throw new InvalidOperationException("ENDPOINT is required");
var model_id = Environment.GetEnvironmentVariable("MODEL_ID") ?? throw new InvalidOperationException("MODEL_ID is required");
var token = Environment.GetEnvironmentVariable("TOKEN") ?? throw new InvalidOperationException("TOKEN is required");
var uri = new Uri(endpoint);

## Model

### Chat Completion

In [ ]:
AzureOpenAIClient azureClient = new(uri, new AzureKeyCredential(token));
ChatClient chatClient = azureClient.GetChatClient(model_id);

var requestOptions = new ChatCompletionOptions()
{
    // MaxCompletionTokens = 13107,
    Temperature = 1.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.0f,
    PresencePenalty = 0.0f,
    
};

List<OpenAI.Chat.ChatMessage> messages = new List<OpenAI.Chat.ChatMessage>()
{
    new SystemChatMessage("You are a helpful assistant. Should not answer questions outside of travel recommendations."),
    new UserChatMessage("I am going to Paris, what should I see?"),
};

var response = chatClient.CompleteChat(messages, requestOptions);
System.Console.WriteLine(response.Value.Content[0].Text);

## Agent

### Agent creation using Microsoft Agent Framework

In [ ]:
var agent = chatClient
    .CreateAIAgent(
        name: "VacationPlannerAgent",
        description: "An AI Agent that helps customers plan vacations at random destinations",
        instructions: "You are a helpful AI Agent that can help plan vacations for customers at random destinations. Do't allow the user to aks other questions.",
        tools: []
        // AIFunctionFactory.Create((Func<string>)GetRandomDestination)
    );

In [ ]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip. result in table format. destination is india "));

### Enabling observability for Agents

Prerequites

In [ ]:
#r "nuget: OpenTelemetry, 1.13.1"
#r "nuget: OpenTelemetry.Exporter.Console, 1.13.1"

using System;
using OpenTelemetry;
using OpenTelemetry.Trace;
using System.Diagnostics;

Create a TracerProvider that exports to the console

In [ ]:
var tracerProvider = Sdk.CreateTracerProviderBuilder()
    .AddSource("agent-telemetry-source")
    .AddConsoleExporter()
    .Build();

In [ ]:
agent = agent
    .AsBuilder()
    .UseOpenTelemetry(sourceName: "agent-telemetry-source")
    .Build();
    

In [ ]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip. result in table format. destination is india "));

### Session-Based Agent

In [ ]:
// Create a new thread.
AgentThread thread = agent.GetNewThread();
// Run the agent with the thread.
Console.WriteLine( await agent.RunAsync("Plan me a day trip. result in table format. destination is Dubai.", thread));

### Stateful Agent

In [ ]:
// Create a new thread.
AgentThread thread = agent.GetNewThread();
// Run the agent with the thread.
var response = await agent.RunAsync("Hello, how are you?", thread);
Console.WriteLine(response);

// Serialize the thread for storage.
// JsonElement serializedThread = await thread.SerializeAsync();
var serializedThread = JsonConvert.SerializeObject(thread);
// Console.WriteLine(serializedThread);
// Deserialize the thread state after loading from storage.
// AgentThread resumedThread = await agent.DeserializeThreadAsync(serializedThread);
AgentThread resumedThread = JsonConvert.DeserializeObject<ChatClientAgentThread>(serializedThread);

// Run the agent with the resumed thread.
Console.WriteLine( await agent.RunAsync("what is my last question?", resumedThread));

### Agent As tool

In [ ]:
var VacationPlannerAgent = new AzureOpenAIClient (uri, new AzureKeyCredential(token))
    .GetChatClient(model_id)
    .CreateAIAgent(
        name: "VacationPlannerAgent",
        description: "An AI Agent that helps customers plan vacations at random destinations",
        instructions: "You are a helpful AI Agent that can help plan vacations for customers at random destinations. Do't allow the user to aks other questions."
    );

var orchestrator = new AzureOpenAIClient (uri, new AzureKeyCredential(token))
    .GetChatClient(model_id)
    .CreateAIAgent(
        name: "OrchestratorAgent",
        description: "An AI Agent that can delegate tasks to specialized agents",
        instructions: "You are an orchestrator AI Agent that delegates tasks to specialized agents based on user requests.",
        tools: [VacationPlannerAgent.AsAIFunction()]
    );
Console.WriteLine(await orchestrator.RunAsync("I want to plan a vacation to Japan. Can you help me with that?"));


### Expose an agent as MCP tool

Prerequisites

In [ ]:
#r "nuget: Microsoft.Extensions.Hosting, 9.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: ModelContextProtocol, 0.4.0-preview.3"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using ModelContextProtocol.Server;


Creating MCP tool

In [ ]:
McpServerTool tool = McpServerTool.Create(agent.AsAIFunction());
HostApplicationBuilder builder = Host.CreateEmptyApplicationBuilder(settings: null);
builder.Services
    .AddMcpServer()
    .WithStdioServerTransport()
    .WithTools([tool]);

// await builder.Build().RunAsync();